In [2]:
# %matplotlib inline
import random
import mindspore as ms
import mindspore.ops as ops
from mindspore.ops import operations as P
from mindspore import Tensor, Parameter, context

from mindspore.common.initializer import initializer, Zero, Normal, Uniform
import mindspore.nn as nn

import numpy as np


In [3]:
def synthetic_data(w, b, num_examples):  #@save
    """生成y=Xw+b+噪声"""
    
    X = ops.normal((num_examples, len(w)), Tensor(0, ms.int32), Tensor(1, ms.int32))
    y = ops.matmul(X, w) + b
    y += ops.normal(y.shape, Tensor(0, ms.int32), Tensor(0.01, ms.float32))
    return X, y.reshape((-1, 1))


In [4]:
def data_iter(batch_size, feature, labels):
    num_examples = len(feature)
    indices = list(range(num_examples))

    # random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        batch_indices = Tensor(
            indices[i: min(i + batch_size, num_examples)])
        yield feature[batch_indices], labels[batch_indices]

## 导入数据及数据预处理


In [5]:
import pandas as pd

 
# 读取Excel文件
excel_file = pd.ExcelFile('data.xls')

# 获取所有表格名称
sheet_names = excel_file.sheet_names

# 导入指定名称的表格数据
if 'Andan' in sheet_names:
    data_andan = excel_file.parse('Andan')
    print("Andan表格数据：")
    print(data_andan)
else:
    print("未找到名称为Andan的表格")

Andan表格数据：
         进水TP   厌氧末端ORP    好氧前段DO   好氧末端TSS      出水pH        温度    出水NH4-N  \
0    3.954938 -572.0870  0.309579  2.426130  8.039920  23.14090   9.370120   
1    4.102314 -572.0870  0.446972  2.430645  8.062150  23.61530   6.456460   
2    4.091968 -572.1510  0.464883  2.430370  8.061620  23.60820   6.395700   
3    4.064541 -572.1510  0.457856  2.410110  8.061665  23.60250   6.366810   
4    4.037114 -572.1510  0.450830  2.389850  8.061710  23.59680   6.337920   
..        ...       ...       ...       ...       ...       ...        ...   
400  4.011535 -570.6770  0.264898  2.232250  8.034375  23.17705   9.868287   
401  4.007578 -570.9330  0.274002  2.333320  8.038460  23.17070  10.006359   
402  4.009888 -571.2535  0.304169  2.376445  8.039115  23.16290  10.256670   
403  4.012198 -571.5740  0.334335  2.419570  8.039770  23.15510  10.506980   
404  3.983568 -571.8305  0.321957  2.422850  8.039845  23.14800  10.407056   

     出水NH4-Npred  
0       9.370120  
1       6.4564

In [6]:
## 提取数据
features_andan = data_andan[['进水TP', '厌氧末端ORP','好氧前段DO', '好氧末端TSS', '出水pH','温度']]
labels = data_andan[['出水NH4-N']]

features_andan= features_andan.to_numpy()
labels = labels.to_numpy()

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import check_random_state

# 假设 features_andan 和 labels 是已有的 numpy 数组

# 划分训练数据和测试数据
random_state = check_random_state(0)
# X_train, X_test, y_train, y_test = train_test_split(features_andan, labels, test_size=0.2, random_state=random_state)
X_train = features_andan[1:330,:]
y_train = labels[1:330,:]
# X_test = features_andan[100:200,:]
# y_test = labels[100:200,:]

X_train2 = features_andan[1:330,:]
y_train2 = labels[1:330,:]
X_test2 = features_andan[100:200,:]
y_test2 = labels[100:200,:]

# 归一化处理
scaler = MinMaxScaler()
yscaler = MinMaxScaler()
X_train_normalized = scaler.fit_transform(X_train)
y_train_normalized = yscaler.fit_transform(y_train)
X_train_normalized2 = scaler.fit_transform(X_train2)
y_train_normalized2 = yscaler.fit_transform(y_train2)
# X_test_normalized = scaler.transform(X_test)
# y_test_normalized = yscaler.transform(y_test)

# 添加噪声到训练数据
noise_std = 0.1  # 噪声标准差
noise = random_state.normal(scale=noise_std, size=X_train_normalized.shape)
y_noise = random_state.normal(scale=noise_std, size=y_train_normalized.shape)
X_train_noisy = X_train_normalized + noise
y_train_nosiy = y_train_normalized + y_noise
noise2 = random_state.normal(scale=noise_std, size=X_train_normalized2.shape)
y_noise2 = random_state.normal(scale=noise_std, size=y_train_normalized2.shape)
X_train_noisy2= X_train_normalized2 + noise2
y_train_nosiy2 = y_train_normalized2 + y_noise2



## 转换格式
X_train = ms.Tensor(X_train_noisy, ms.float32)
y_train = ms.Tensor(y_train_nosiy, ms.float32)

X_train2 = ms.Tensor(X_train_noisy2, ms.float32)
y_train2= ms.Tensor(y_train_nosiy2, ms.float32)

# X_test = ms.Tensor(X_test_normalized, ms.float32)
# y_test = ms.Tensor(y_test_normalized, ms.float32)


In [7]:
from mindspore import dataset

class MyDataset(dataset.Dataset):
    def __init__(self, data, labels):
        super(MyDataset, self).__init__()
        self.data = data
        self.labels = labels

    def __getitem__(self, index):
        return self.data[index], self.labels[index]

    def __len__(self):
        return len(self.data)

## 定义网络

In [9]:
import mindspore.numpy as np

context.set_context(mode=context.GRAPH_MODE, device_target="CPU")       
## 定义FNN        
class FNN(nn.Cell):
    def __init__(self, InDim, RuleNum, OutDim):
        super(FNN, self).__init__()
        self.exp = ops.Exp()
        self.prod = ops.ReduceProd(keep_dims=True)
        self.sum = ops.ReduceSum(keep_dims=True)
        self.matmul = P.MatMul()
        self.sub = ops.Sub()
        self.square = ops.Square()
        self.div = ops.RealDiv()
        self.mul = ops.Mul()
        self.expand_dims = P.ExpandDims()
        self.transpose = P.Transpose()
        self.InDim = InDim
        self.RuleNum = RuleNum
        self.OutDim = OutDim
        self.Center = Parameter(initializer('normal', [InDim, RuleNum]), name='center')
        self.Width = Parameter(initializer('ones', [InDim, RuleNum]), name='width')
        self.W = Parameter(initializer('normal', [RuleNum, OutDim]), name='W')

    def construct(self, SamIn):
        
        
        MemFunUnitOut = self.exp(self.div(self.square(self.sub(np.broadcast_to(self.transpose(SamIn, (1,0)), (self.InDim, self.RuleNum)), self.Center)), self.square(self.Width)))
        # print(f"MemFunUnitOut:{MemFunUnitOut}")
        #MemFunUnitOut = self.exp(self.div(self.square(self.sub(SamIn, self.transpose(self.Center, (1,0)))), self.square(self.transpose(self.Width, (1,0)))))
        RuleUnitOut = self.prod(MemFunUnitOut,0)
        # print(f"RuleUnitOut:{RuleUnitOut}")
        RuleUnitOutSum = self.sum(RuleUnitOut)
        # print(f" RuleUnitOutSum :{RuleUnitOutSum}")
        NormValue = self.div(RuleUnitOut, RuleUnitOutSum)
        # print(f"NormValue.shape:{NormValue}")
        NetOut = self.matmul(NormValue, self.transpose(self.W, (0,1)))
        # print(f"c:{ self.Center}")
        # print(f"NetOut.shape:{NetOut.shape}")
        
        
        
        return NetOut

In [10]:
class WithLossCell(nn.Cell):
    def __init__(self, network, loss):
        super(WithLossCell, self).__init__(auto_prefix=False)
        self.network = network
        self.loss = loss

    def construct(self, data, label):
        predict = self.network(data)
        return self.loss(predict, label)

class FNNTrain:
    def __init__(self, InDim, RuleNum, OutDim, lr, MaxEpoch, E0):
        self.network = FNN(InDim, RuleNum, OutDim)
        self.loss = nn.MSELoss()
        self.optimizer = nn.SGD(self.network.trainable_params(), learning_rate=lr)
        self.net_with_loss = WithLossCell(self.network, self.loss)
        self.train_network = nn.TrainOneStepCell(self.net_with_loss, self.optimizer)
        self.train_network.set_train()
        self.MaxEpoch = MaxEpoch
        self.E0 = E0
        self.losslist = [] 
        
    
    def run(self, fe, out):
        # epoch = 0
        for epoch in range(0,self.MaxEpoch):
            print(epoch)
            a=0
            train_dataset = data_iter(1, fe, out)
            
        # while epoch < self.MaxEpoch:
            # print(f"迭代之前：{self.network.W.data.asnumpy()}")
            for i, (data, label) in enumerate(train_dataset):
            # for columns in dataset.create_dict_iterator():
                a=a+1
                # print(a)
                # print(f'f:{i}')
                # data = columns['data']
                # label = columns['label']
                loss = self.train_network(data, label)
                # loss.backward()
                # print(f"epoch: {epoch}/{self.MaxEpoch}, loss: {loss}")
            # print(f"迭代之后：{self.network.W.data.asnumpy()}")
            self.losslist.append(loss.asnumpy())
            # epoch += 1
            
            print(f'epoch_{epoch}: {loss.asnumpy()}')
            # if loss.asnumpy() < self.E0:
            #     break
            ms.save_checkpoint(self.network, "./MyNet.ckpt")
        return self.network

In [11]:
class WithLossCell(nn.Cell):
    def __init__(self, network, loss):
        super(WithLossCell, self).__init__(auto_prefix=False)
        self.network = network
        self.loss = loss

    def construct(self, data, label):
        predict = self.network(data)
        return self.loss(predict, label)

class FNNTrain2:
    def __init__(self, InDim, RuleNum, OutDim, lr, MaxEpoch, E0):
        self.network = FNN(InDim, RuleNum, OutDim)
        self.loss = nn.MSELoss()
        self.optimizer = nn.SGD(self.network.trainable_params(), learning_rate=lr)
        self.net_with_loss = WithLossCell(self.network, self.loss)
        self.train_network = nn.TrainOneStepCell(self.net_with_loss, self.optimizer)
        self.train_network.set_train()
        self.MaxEpoch = MaxEpoch
        self.E0 = E0
        self.losslist = [] 
        
    
    def run(self, fe, out):
        # epoch = 0
        for epoch in range(0,self.MaxEpoch):
            print(epoch)
            a=0
            train_dataset = data_iter(1, fe, out)
            
        # while epoch < self.MaxEpoch:
            # print(f"迭代之前：{self.network.W.data.asnumpy()}")
            for i, (data, label) in enumerate(train_dataset):
            # for columns in dataset.create_dict_iterator():
                a=a+1
                # print(a)
                # print(f'f:{i}')
                # data = columns['data']
                # label = columns['label']
                loss = self.train_network(data, label)
                # loss.backward()
            print(f"epoch: {epoch}/{self.MaxEpoch}, loss: {loss}")
            # print(f"迭代之后：{self.network.W.data.asnumpy()}")
            self.losslist.append(loss.asnumpy())
            # epoch += 1
            
            print(f'epoch_{epoch}: {loss.asnumpy()}')
            # if loss.asnumpy() < self.E0:
            #     break
            ms.save_checkpoint(self.network, "./MyNet2.ckpt")
        return self.network

## 训练

In [12]:
TrainSamInN = Tensor(X_train, ms.float32)  
TrainSamOutN = Tensor(y_train, ms.float32)
TrainSamInN2 = Tensor(X_train2, ms.float32)  
TrainSamOutN2 = Tensor(y_train2, ms.float32)

print(TrainSamInN.shape)
print(TrainSamOutN.shape)
print(y_train.shape)

(329, 6)
(329, 1)
(329, 1)


In [14]:
TrainSamInN = Tensor(X_train, ms.float32)  
TrainSamOutN = Tensor(y_train, ms.float32)
TrainSamInN2 = Tensor(X_train2, ms.float32)  
TrainSamOutN2 = Tensor(y_train2, ms.float32)

# data_loader = data_iter(1, TrainSamInN, TrainSamOutN)

fnn_train = FNNTrain(InDim=6, RuleNum=15, OutDim=1, lr=0.05, MaxEpoch=100, E0=0.001)
network = fnn_train.run(TrainSamInN, TrainSamOutN)

fnn_train2 = FNNTrain2(InDim=6, RuleNum=15, OutDim=1, lr=0.05, MaxEpoch=100, E0=0.001)
network2 = fnn_train2.run(TrainSamInN2, TrainSamOutN2)

# import matplotlib.pyplot as plt

# plt.plot(fnn_train.losslist)

0
epoch_0: 0.4347507953643799
1
epoch_1: 0.380573570728302
2
epoch_2: 0.3748275339603424
3
epoch_3: 0.37418439984321594
4
epoch_4: 0.3740886449813843
5
epoch_5: 0.3740313649177551
6
epoch_6: 0.37393781542778015
7
epoch_7: 0.3737626075744629
8
epoch_8: 0.3734153211116791
9
epoch_9: 0.3726392090320587
10
epoch_10: 0.3700118958950043
11
epoch_11: 0.3158557116985321
12
epoch_12: 0.25144922733306885
13
epoch_13: 0.21812647581100464
14
epoch_14: 0.19502858817577362
15
epoch_15: 0.17869986593723297
16
epoch_16: 0.16795963048934937
17
epoch_17: 0.1609029471874237
18
epoch_18: 0.15583668649196625
19
epoch_19: 0.1525530219078064
20
epoch_20: 0.1505507230758667
21
epoch_21: 0.14927685260772705
22
epoch_22: 0.14858296513557434
23
epoch_23: 0.14799340069293976
24
epoch_24: 0.14729426801204681
25
epoch_25: 0.14655759930610657
26
epoch_26: 0.14583364129066467
27
epoch_27: 0.14514273405075073
28
epoch_28: 0.14449115097522736
29
epoch_29: 0.14387843012809753
30
epoch_30: 0.1433015912771225
31
epoch_31:

In [49]:
losss = fnn_train.losslist

In [70]:
class FNNTest:
    def __init__(self, InDim, RuleNum, OutDim):
        self.network = FNN(InDim, RuleNum, OutDim)
        
        
        
    
    def run(self, fe, out):
        testloader = data_iter(1, fe, out)
        
        all_loss = []
        # net = FNN(InDim=6, RuleNum=15, OutDim=1)
        param_dict = ms.load_checkpoint("MyNet.ckpt")
        print(param_dict)
        ms.load_param_into_net(self.network, param_dict)

        for i, (x, y) in enumerate(testloader):
            print(f'x.shpae:{x.shape}; y.shape:{y.shape}')
            pred = self.network(x)
            print(f'pred:{pred}')
            loss = y - pred
            all_loss.append(loss)
            print(f"loss:{loss}\n")

        print(f'average loss:{sum(loss)/len(loss)}') 

In [74]:
TestSamInN = Tensor(X_test, ms.float32)  
TestSamOutN = Tensor(y_test, ms.float32)
print(TestSamInN.shape)
print(TestSamOutN.shape)

fnn_test = FNNTest(InDim=6, RuleNum=15, OutDim=1)
network = fnn_test.run(TestSamInN, TestSamOutN)

(100, 6)
(100, 1)
{'center': Parameter (name=center, shape=(6, 15), dtype=Float32, requires_grad=True), 'width': Parameter (name=width, shape=(6, 15), dtype=Float32, requires_grad=True), 'W': Parameter (name=W, shape=(15, 1), dtype=Float32, requires_grad=True)}
x.shpae:(1, 6); y.shape:(1, 1)
pred:[[0.6841784]]
loss:[[8.151142]]

x.shpae:(1, 6); y.shape:(1, 1)
pred:[[0.6841784]]
loss:[[8.128117]]

x.shpae:(1, 6); y.shape:(1, 1)
pred:[[0.6841784]]
loss:[[8.105092]]

x.shpae:(1, 6); y.shape:(1, 1)
pred:[[0.6841784]]
loss:[[8.133282]]

x.shpae:(1, 6); y.shape:(1, 1)
pred:[[0.6841784]]
loss:[[8.161471]]

x.shpae:(1, 6); y.shape:(1, 1)
pred:[[0.6841784]]
loss:[[8.286992]]

x.shpae:(1, 6); y.shape:(1, 1)
pred:[[0.6841784]]
loss:[[8.412512]]

x.shpae:(1, 6); y.shape:(1, 1)
pred:[[0.6841784]]
loss:[[8.640067]]

x.shpae:(1, 6); y.shape:(1, 1)
pred:[[0.6841784]]
loss:[[8.867621]]

x.shpae:(1, 6); y.shape:(1, 1)
pred:[[0.6841784]]
loss:[[8.776782]]

x.shpae:(1, 6); y.shape:(1, 1)
pred:[[0.6841784]

[ERROR] CORE(5394,7fc18df0e340,python):2023-07-24-21:58:14.318.884 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_5394/822726262.py]
[WARNING] ANALYZER(5394,7fc18df0e340,python):2023-07-24-21:58:14.321.084 [mindspore/ccsrc/pipeline/jit/static_analysis/prim.cc:2551] CheckInterpretInput] When using JIT Fallback to handle script 'np.broadcast_to(self.transpose(SamIn, (1,0)), (self.InDim, self.RuleNum))', found variable 'SamIn' to be a tensor.
[ERROR] CORE(5394,7fc18df0e340,python):2023-07-24-21:58:14.342.274 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_5394/822726262.py]
[WARNING] ANALYZER(5394,7fc18df0e340,python):2023-07-24-21:58:14.344.298 [mindspore/ccsrc/pipeline/jit/static_analysis/prim.cc:2551] CheckInterpretInput] When using JIT Fallback to handle script 'np.broadcast_to(self.transpose(SamIn, (1,0)), (self.InDim, self.RuleNum))', found variable 'SamIn' to be a tensor.


pred:[[0.6841784]]
loss:[[8.105092]]

x.shpae:(1, 6); y.shape:(1, 1)
pred:[[0.6841784]]
loss:[[8.133282]]

x.shpae:(1, 6); y.shape:(1, 1)
pred:[[0.6841784]]
loss:[[8.161471]]

x.shpae:(1, 6); y.shape:(1, 1)
pred:[[0.6841784]]
loss:[[8.286992]]

x.shpae:(1, 6); y.shape:(1, 1)
pred:[[0.6841784]]
loss:[[8.412512]]

x.shpae:(1, 6); y.shape:(1, 1)
pred:[[0.6841784]]
loss:[[8.640067]]

x.shpae:(1, 6); y.shape:(1, 1)
pred:[[0.6841784]]
loss:[[8.867621]]

x.shpae:(1, 6); y.shape:(1, 1)
pred:[[0.6841784]]
loss:[[8.776782]]

x.shpae:(1, 6); y.shape:(1, 1)
pred:[[0.6841784]]
loss:[[8.685942]]

x.shpae:(1, 6); y.shape:(1, 1)
pred:[[0.6841784]]
loss:[[5.7722816]]

x.shpae:(1, 6); y.shape:(1, 1)
pred:[[0.6841784]]
loss:[[5.7115216]]

x.shpae:(1, 6); y.shape:(1, 1)
pred:[[0.6841784]]
loss:[[5.6826315]]

x.shpae:(1, 6); y.shape:(1, 1)
pred:[[0.6841784]]
loss:[[5.653742]]

x.shpae:(1, 6); y.shape:(1, 1)
pred:[[0.6841784]]
loss:[[5.6355867]]

x.shpae:(1, 6); y.shape:(1, 1)
pred:[[0.6841784]]
loss:[[8.05

In [19]:
TestSamInN = Tensor(X_test, ms.float32)  
TestSamOutN = Tensor(y_test, ms.float32)

network = FNN(InDim=6, RuleNum=15, OutDim=1)
param_dict = ms.load_checkpoint("MyNet.ckpt")
print(param_dict)
ms.load_param_into_net(network, param_dict)


testloader = data_iter(1, TestSamInN, TestSamOutN)

all_loss = []

for i, (x, y) in enumerate(testloader):
    print(f'x.shpae:{x.shape}; y.shape:{y.shape}')
    pred = network(x)
    print(f'pred_shape:{pred.shape}')
    loss = y - pred
    all_loss.append(loss)
    print(f"loss:{loss}\n")

print(f'average loss:{sum(loss)/len(loss)}') 

{'center': Parameter (name=center, shape=(6, 15), dtype=Float32, requires_grad=True), 'width': Parameter (name=width, shape=(6, 15), dtype=Float32, requires_grad=True), 'W': Parameter (name=W, shape=(15, 1), dtype=Float32, requires_grad=True)}
x.shpae:(1, 6); y.shape:(1, 1)
[[nan]]
pred_shape:(1, 1)
loss:[[nan]]

x.shpae:(1, 6); y.shape:(1, 1)
[[nan]]
pred_shape:(1, 1)
loss:[[nan]]

x.shpae:(1, 6); y.shape:(1, 1)
[[nan]]
pred_shape:(1, 1)
loss:[[nan]]

x.shpae:(1, 6); y.shape:(1, 1)
[[nan]]
pred_shape:(1, 1)
loss:[[nan]]

x.shpae:(1, 6); y.shape:(1, 1)
[[nan]]
pred_shape:(1, 1)
loss:[[nan]]

x.shpae:(1, 6); y.shape:(1, 1)
[[nan]]
pred_shape:(1, 1)
loss:[[nan]]

x.shpae:(1, 6); y.shape:(1, 1)
[[nan]]
pred_shape:(1, 1)
loss:[[nan]]

x.shpae:(1, 6); y.shape:(1, 1)
[[nan]]
pred_shape:(1, 1)
loss:[[nan]]

x.shpae:(1, 6); y.shape:(1, 1)
[[nan]]
pred_shape:(1, 1)
loss:[[nan]]

x.shpae:(1, 6); y.shape:(1, 1)
[[nan]]
pred_shape:(1, 1)
loss:[[nan]]

x.shpae:(1, 6); y.shape:(1, 1)
[[nan]]
pred_sh

[ERROR] CORE(5394,7fc18df0e340,python):2023-07-22-14:55:39.962.421 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_5394/822726262.py]
[WARNING] ANALYZER(5394,7fc18df0e340,python):2023-07-22-14:55:39.964.487 [mindspore/ccsrc/pipeline/jit/static_analysis/prim.cc:2551] CheckInterpretInput] When using JIT Fallback to handle script 'np.broadcast_to(self.transpose(SamIn, (1,0)), (self.InDim, self.RuleNum))', found variable 'SamIn' to be a tensor.
[ERROR] CORE(5394,7fc18df0e340,python):2023-07-22-14:55:39.987.739 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_5394/822726262.py]
[WARNING] ANALYZER(5394,7fc18df0e340,python):2023-07-22-14:55:39.989.658 [mindspore/ccsrc/pipeline/jit/static_analysis/prim.cc:2551] CheckInterpretInput] When using JIT Fallback to handle script 'np.broadcast_to(self.transpose(SamIn, (1,0)), (self.InDim, self.RuleNum))', found variable 'SamIn' to be a tensor.


x.shpae:(1, 6); y.shape:(1, 1)
[[nan]]
pred_shape:(1, 1)
loss:[[nan]]

x.shpae:(1, 6); y.shape:(1, 1)
[[nan]]
pred_shape:(1, 1)
loss:[[nan]]

x.shpae:(1, 6); y.shape:(1, 1)
[[nan]]
pred_shape:(1, 1)
loss:[[nan]]

x.shpae:(1, 6); y.shape:(1, 1)
[[nan]]
pred_shape:(1, 1)
loss:[[nan]]

x.shpae:(1, 6); y.shape:(1, 1)
[[nan]]
pred_shape:(1, 1)
loss:[[nan]]

x.shpae:(1, 6); y.shape:(1, 1)
[[nan]]
pred_shape:(1, 1)
loss:[[nan]]

x.shpae:(1, 6); y.shape:(1, 1)
[[nan]]
pred_shape:(1, 1)
loss:[[nan]]

x.shpae:(1, 6); y.shape:(1, 1)
[[nan]]
pred_shape:(1, 1)
loss:[[nan]]

x.shpae:(1, 6); y.shape:(1, 1)
[[nan]]
pred_shape:(1, 1)
loss:[[nan]]

x.shpae:(1, 6); y.shape:(1, 1)
[[nan]]
pred_shape:(1, 1)
loss:[[nan]]

x.shpae:(1, 6); y.shape:(1, 1)
[[nan]]
pred_shape:(1, 1)
loss:[[nan]]

x.shpae:(1, 6); y.shape:(1, 1)
[[nan]]
pred_shape:(1, 1)
loss:[[nan]]

x.shpae:(1, 6); y.shape:(1, 1)
[[nan]]
pred_shape:(1, 1)
loss:[[nan]]

x.shpae:(1, 6); y.shape:(1, 1)
[[nan]]
pred_shape:(1, 1)
loss:[[nan]]

x.shpa

## Test

In [ ]:
import mindspore.numpy as np
sub = ops.Sub()
transpose = P.Transpose()
a = np.rand((1,6))
b = np.rand((6,15))
sub(
    transpose(a, (1,0))
    -b)

In [ ]:
np.broadcast_to(transpose(a, (1,0)), (6,15))

In [ ]:
np.broadcast_to(transpose(a, (1,0)), (6,15)) - b

In [ ]:
sum = ops.ReduceSum()
a = np.rand((1,6))
print(sum(a, 1))